# Nimbus ideal vs. payoff ideal

There is some issues that cause the results attained by Nimbus method to be higher than the ideal values attained by calculating the payoff table for the clusters.

In [1]:
from interactiveBoreal import *
import gradutil
from BorealWeights import BorealWeightedProblem

In [19]:
kehys = ReferenceFrame()
nclust = 150
kehys.cluster(nclust=nclust, seedn=2, outdata=kehys.x_stack)
data = kehys.centers
nvar = len(kehys.x_norm)
weights_norm = kehys.weights/nvar
ideal = kehys.ideal
nadir = kehys.nadir

In [3]:
solver_name = 'cplex'

Now we try to calculate ideal and possible nadir values for all the objectives using nimbus scalarization:

In [24]:
nimbus_ref = np.array([[kehys.ideal[0], kehys.nadir[1], kehys.nadir[2], kehys.nadir[3]],
                       [kehys.nadir[0], kehys.ideal[1], kehys.nadir[2], kehys.nadir[3]],
                       [kehys.nadir[0], kehys.nadir[1], kehys.ideal[2], kehys.nadir[3]],
                       [kehys.nadir[0], kehys.nadir[1], kehys.nadir[2], kehys.ideal[3]]])
stay = np.array([], dtype=int)
detoriate = np.array([], dtype=int)
nimbus_res_cluster = []
nimbus_res_orig = []
nimbus_problems = []
for i in range(len(nimbus_ref)):
    minmax = np.array([i], dtype=int)
    nimbus_problems.append(NIMBUS(kehys.ideal, kehys.nadir, nimbus_ref[i], kehys.centers, 
                                  minmax, stay, detoriate, np.array([0,0,0,0]), 
                                  weights=kehys.weights, nvar=nvar))
    nimbus_solver = Solver(nimbus_problems[i].model, solver=solver_name)
    res = nimbus_solver.solve() 
    nimbus_res_cluster.append([gradutil.model_to_real_values(kehys.x_stack[:,:,j], 
                                                             nimbus_problems[i].model, 
                                                             kehys.xtoc) 
                               for j in range(len(nimbus_ref))])
    nimbus_res_orig.append([gradutil.cluster_to_value(kehys.x_stack[:,:,j], gradutil.res_to_list(nimbus_problems[i].model), kehys.weights)
                            for j in range(len(nimbus_ref))])
nimbus_res = np.array(nimbus_res)

In [25]:
for k in nimbus_res_cluster:
    print(k)

As we can see, the greatest values column wise are at the diagonal. As NIMBUS uses exactly these in its operations this seems very good!

In [26]:
for k in nimbus_res_orig:
    print(k)

Now when we print the original variable values, we see that the biggest values are not anymore at the diagonal. This was expected so no need to worry

However the ideal of the attained results is:


In [29]:
nimbus_ideal = np.max(nimbus_res_cluster, axis=0)
nimbus_ideal

And the nadir

In [30]:
nimbus_nadir = np.min(nimbus_res_cluster, axis=0)
nimbus_nadir

However if we are using payoff-table for the same data, the results are different:

In [48]:
data = kehys.centers
solver = SolverFactory(solver_name)
problems = []
for i in range(np.shape(data)[-1]):
    problems.append(BorealWeightedProblem(data[:, :, i], kehys.weights, nvar))

for j in range(len(problems)):
    solver.solve(problems[j].model)

payoff = [[gradutil.cluster_to_value(kehys.x_stack[:,:,i], 
                                      gradutil.res_to_list(nimbus_problems[j].model), 
                                      kehys.weights)
            for i in range(np.shape(data)[-1])]
           for j in range(len(problems))]

In [49]:
for p in payoff:
    print(p)

Lets see if here is something similar going on than with the nimbus ideals

In [50]:
for p in nimbus_problems:
    print(gradutil.cluster_to_value(kehys.x_stack, gradutil.res_to_list(p.model), weights_norm))

It almost looks like that we are minimizing the last objectives...

In [51]:
payoff_ideal = np.max(payoff, axis=0)
payoff_ideal

In [52]:
payoff_nadir = np.min(payoff, axis=0)
payoff_nadir

Compare the differences also:


In [53]:
payoff_ideal - nimbus_ideal

In [54]:
payoff_nadir - nimbus_nadir

Absolutely the values are quite big, but the directions of the differences are good.

Relative differences:

In [16]:
(payoff_ideal - nimbus_ideal)/payoff_ideal

In [17]:
(payoff_nadir - nimbus_nadir)/payoff_nadir

The differences are much bigger for the nadir, but it is no surprise because of the payoff table -method

The ideals are much closer so probably there is no need to worry. There used to be greater differences, but setting the closest point as the centroid helped a lot and also the payoff table had to be fixed a bit.

We also have own method to calculate nadir and ideal using payoff table and just the cluster data:

In [18]:
gradutil.calc_ideal_n_nadir(kehys.centers, kehys.xtoc, kehys.weights)

As we can see, these are again different results because we are using just the data from the clusters and not mapping them back to the original values.